In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q evaluate
!pip install -q parsivar
!pip install -q farm-haystack[colab,inference,metrics,elasticsearch,preprocessing]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall -q transformers

Proceed (Y/n)? y


In [ ]:
!pip install --no-cache-dir -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm-haystack 1.25.5 requires transformers==4.39.3, but you have transformers 4.41.1 which is incompatible.


In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
from haystack.utils import launch_es
import time

launch_es()
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

doc_index = "hybrid_docs"
label_index = "hybrid_labels"

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [ ]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(
    split_by="word",
    language = 'fa',
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.

# docs_to_index = preprocessor.process(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

doc_index = "hybrid_docs"
label_index = "hybrid_labels"

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [ ]:
from haystack.nodes import DensePassageRetriever, BM25Retriever

sparse_retriever = BM25Retriever(document_store=document_store)
dense_retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="/content/drive/MyDrive/saved_models/dpr_parsbert/query_encoder",
                                  passage_embedding_model="/content/drive/MyDrive/saved_models/dpr_parsbert/passage_encoder",
                                  use_gpu=True,
                                  max_seq_len_passage=256,
                                  embed_title=True)


turn test set into squad format

In [ ]:
import json

with open('/content/drive/MyDrive/dev_IR-10_percent.json', 'r', encoding = 'utf-8-sig') as f:
  test_data = json.load(f)

new_dict_data_list = []
for data in test_data:
  new_data = {}
  new_data['title'] = ''
  new_data['paragraphs'] = [{'context': data['context'], 'qas':[{'answers':[{'answer_start':data['answers'][0]['start_char'], 'text':data['answers'][0]['text']}],'id':data['pq_id'], 'question': data['question'], 'is_impossible': False}]}]

  new_dict_data_list.append(new_data)

final_data = {}
# random.shuffle(new_dict_data_list)
final_data['data'] = new_dict_data_list

with open('/content/test.json', 'w') as fp:
  json.dump(final_data, fp,)

In [ ]:
import json

# with open('/content/drive/MyDrive/temporary_test.json' , 'r', encoding = 'utf-8-sig') as f:
#   test_data = json.load(f)

# with open('/content/test.json' , 'w') as f:
#    json.dump(test_data, f)

document_store.delete_documents()
# document_store.write_documents(docs_to_index)
document_store.add_eval_data(
    filename="/content/test.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2874.78docs/s]
There were conversion errors for question ids: ['s7.85', 's2.215']


In [ ]:
document_store.update_embeddings(retriever=dense_retriever, index=doc_index)

Create embeddings: 100%|██████████| 464/464 [00:06<00:00, 89.22 Docs/s]
Updating embeddings: 10000 Docs [00:10, 928.79 Docs/s]


In [ ]:
from haystack.nodes import JoinDocuments, SentenceTransformersRanker

join_documents = JoinDocuments(join_mode="concatenate")
# rerank = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-6-v2")
# rerank = SentenceTransformersRanker(model_name_or_path="/content/drive/MyDrive/cross_encoder/cross_encoder-2024-02-10_10-21-21")
rerank = SentenceTransformersRanker(model_name_or_path="NeginShams/cross_encoder_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="NeginShams/mbert-Quran_QA", top_k=4, use_gpu = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/667M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
from haystack.pipelines import Pipeline

pipeline = Pipeline()
pipeline.add_node(component=sparse_retriever, name="SparseRetriever", inputs=["Query"])
pipeline.add_node(component=dense_retriever, name="DenseRetriever", inputs=["Query"])
pipeline.add_node(component=join_documents, name="JoinDocuments", inputs=["SparseRetriever", "DenseRetriever"])
pipeline.add_node(component=rerank, name="ReRanker", inputs=["JoinDocuments"])
pipeline.add_node(component=reader, name="Reader", inputs=["ReRanker"])

In [ ]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)

In [ ]:
eval_result = pipeline.eval(labels=eval_labels, params={"ReRanker": {"top_k": 5}})

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 14.61 Batches/s]


In [ ]:
# retriever_result = eval_result["Retriever"]
retriever_result = eval_result["ReRanker"]
reader_result = eval_result["Reader"]

eval_result.save("../")

In [ ]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["ReRanker"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["ReRanker"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["ReRanker"]["mrr"]}')
print(f'Retriever - Precision: {metrics["ReRanker"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["ReRanker"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.9022403258655805
Retriever - Recall (multiple relevant documents): 0.8998642226748134
Retriever - Mean Reciprocal Rank: 0.7417854718262051
Retriever - Precision: 0.25010183299389005
Retriever - Mean Average Precision: 0.720397714415026
Reader - F1-Score: 0.894646140586242
Reader - Exact Match: 0.8676171079429735


In [ ]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

KeyError: 'Retriever'